# Tarea 1:

### Alumno: Nicolas Silva Andujar
### Código: 20200832

#### Ejercicio 1: Realizar los gráficos por semana en vez de años

Para este ejercicio, he seleccionado el año 2016, puesto que es un periodo con gran afluencia de casos dentro de los años disponibles que brinda la data

Vamos a cargar la data, limpiarla y configurarla para la realización de los gráficos

In [3]:
import pandas as pd
data= "https://github.com/SocialAnalytics-StrategicIntelligence/TableOperations/raw/main/dengue_ok.pkl"

dengue =pd.read_pickle(data)
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501236 entries, 0 to 501235
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   departamento  501236 non-null  object  
 1   provincia     501236 non-null  object  
 2   distrito      501236 non-null  object  
 3   ano           501236 non-null  int64   
 4   semana        501236 non-null  int64   
 5   sexo          501236 non-null  object  
 6   edad          501236 non-null  int64   
 7   enfermedad    501236 non-null  category
 8   case          501236 non-null  int64   
dtypes: category(1), int64(4), object(4)
memory usage: 31.1+ MB


In [4]:
dengue['enfermedad_text']=dengue.enfermedad.astype(str)

dengue.replace({'enfermedad_text':{'SIN_SEÑALES':'1_SIN_SEÑALES','ALARMA':'2_ALARMA','GRAVE':'3_GRAVE'}},inplace=True)

In [5]:
cond_1 =dengue["ano"] == 2016 #Seleccionamos el año 2016
dengue = dengue[cond_1]

In [7]:
dengue["enfermedad"].value_counts()

enfermedad
SIN_SEÑALES    21656
ALARMA          3381
GRAVE            123
Name: count, dtype: int64

In [8]:
dengue["enfermedad_text"] = dengue.enfermedad.astype(str)
dengue.replace({'enfermedad_text':{'SIN_SEÑALES':'1_SIN_SEÑALES','ALARMA':'2_ALARMA','GRAVE':'3_GRAVE'}},inplace=True)

In [9]:
dengue.semana.value_counts(sort=False)

semana
5      599
34      84
4      473
3      451
2      417
10     933
1      307
45     214
39     122
31     195
21     877
18    1120
15    1200
14    1138
13     879
12     729
9      895
7      769
8      822
50     165
52     155
51     136
49     188
20    1049
48     197
44     183
47     245
6      766
22     821
29     192
28     261
27     337
26     356
43     129
19    1130
17    1060
16    1181
41     202
42     207
46     210
36     100
32     181
33     128
23     677
30     168
11     891
24     641
40     156
37     110
38     125
25     480
35     109
Name: count, dtype: int64

Creamos grupos etarios que serán utilizados posteriormente

In [10]:
binLimits=[0,15,50,110]
theLabels=["a_menor_a_16","b_entre_16y50","c_mayor_a_50"]
dengue["edad_grupos"]=pd.cut(dengue['edad'], include_lowest=True,
                                     bins=binLimits, 
                                     labels=theLabels,
                                     ordered=True)

# see

dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,case,enfermedad_text,edad_grupos
231873,LORETO,MAYNAS,IQUITOS,2016,5,M,50,ALARMA,1,2_ALARMA,b_entre_16y50
231874,UCAYALI,CORONEL PORTILLO,YARINACOCHA,2016,5,M,65,SIN_SEÑALES,1,1_SIN_SEÑALES,c_mayor_a_50
231875,LORETO,MAYNAS,IQUITOS,2016,5,F,8,ALARMA,1,2_ALARMA,a_menor_a_16
231876,LORETO,REQUENA,REQUENA,2016,5,M,38,SIN_SEÑALES,1,1_SIN_SEÑALES,b_entre_16y50
231877,LORETO,MAYNAS,PUNCHANA,2016,5,M,11,SIN_SEÑALES,1,1_SIN_SEÑALES,a_menor_a_16


Importamos altair para los gráficos

In [11]:
#!pip install altair -U
#!pip install "vegafusion-jupyter[embed]"
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

Examinamos la intensidad de los casos de dengue en las semanas del 2016

In [12]:
grafico_intensidad = alt.Chart(dengue)
enc_dengue = grafico_intensidad.encode(
    x="semana:Q",
    y="mean(edad):Q",
    color="enfermedad_text:N",
)
enc_dengue.mark_line() + enc_dengue.mark_errorband()

alt.LayerChart(...)

Ahora por grupos etarios: Cantidad de casos en las semanas del 2016 por grupo etario

In [13]:
enc_dengue=grafico_intensidad.encode(
    x='semana:Q',
    y=alt.Y('sum(case):Q'),
    color='enfermedad_text:N',
    tooltip=['sum(case):Q','semana:Q']
).interactive()

enc_dengue.mark_line().facet(
    row='sexo:N',
    column='edad_grupos:N'
) 

alt.FacetChart(...)

Ahora lo apreciamos con logaritmos:

In [14]:
enc_dengue =grafico_intensidad.encode(
    x="semana:Q",
    y=alt.Y("sum(case):Q", scale =alt.Scale(type="log")),
    color ="enfermedad_text:N",
    tooltip = ["sum(case):Q","semana:Q"]
).interactive()

enc_dengue.mark_point().facet(
    row="sexo:N",
    column="edad_grupos:N")

alt.FacetChart(...)

Ahora expresaremos los resultados en tablas:

In [15]:
indexList =["edad_grupos","semana","sexo","enfermedad_text"]
aggregator ={'edad': ['median',"mean","min","max"]}
LevelByWeek_statsFull =dengue.groupby(indexList,observed =True).agg(aggregator)
LevelByWeek_statsFull

edad                   
                                         median       mean min max
edad_grupos  semana sexo enfermedad_text                          
a_menor_a_16 1      F    1_SIN_SEÑALES     10.0   9.480000   2  15
                         2_ALARMA          13.0  10.750000   4  13
                         3_GRAVE            3.0   3.000000   3   3
                    M    1_SIN_SEÑALES     11.0   9.363636   1  15
                         2_ALARMA           7.0   8.000000   5  12
...                                         ...        ...  ..  ..
c_mayor_a_50 51     M    2_ALARMA          62.0  62.000000  62  62
             52     F    1_SIN_SEÑALES     61.0  62.500000  54  76
                         2_ALARMA          65.0  67.000000  52  84
                    M    1_SIN_SEÑALES     58.5  61.250000  53  74
                         2_ALARMA          68.5  68.500000  61  76

[705 rows x 4 columns]

In [16]:
LevelByWeek_statsFull.stack(future_stack=True)

edad
edad_grupos  semana sexo enfermedad_text              
a_menor_a_16 1      F    1_SIN_SEÑALES   median  10.00
                                         mean     9.48
                                         min      2.00
                                         max     15.00
                         2_ALARMA        median  13.00
...                                                ...
c_mayor_a_50 52     M    1_SIN_SEÑALES   max     74.00
                         2_ALARMA        median  68.50
                                         mean    68.50
                                         min     61.00
                                         max     76.00

[2820 rows x 1 columns]

Ahora a nivel departamental y provincia:

In [17]:
indexList=['semana','departamento','provincia','enfermedad_text']
aggregator={'case':['sum']}
ByWeekPlace=dengue.groupby(indexList,observed=True).agg(aggregator)
ByWeekPlace

case
                                                      sum
semana departamento provincia        enfermedad_text     
1      AMAZONAS     UTCUBAMBA        2_ALARMA           1
       ANCASH       CASMA            1_SIN_SEÑALES      2
       AYACUCHO     HUANTA           1_SIN_SEÑALES      7
                    LA MAR           1_SIN_SEÑALES     53
                                     2_ALARMA           1
...                                                   ...
52     SAN MARTIN   TOCACHE          1_SIN_SEÑALES      4
                                     2_ALARMA           3
       UCAYALI      ATALAYA          2_ALARMA           1
                    CORONEL PORTILLO 1_SIN_SEÑALES     19
                                     2_ALARMA          17

[2367 rows x 1 columns]

Eliminamos los valores perdidos y cambiamos la dirección de la tabla

In [18]:
ByWeekPlace_wide =ByWeekPlace.unstack().fillna(0)
ByWeekPlace_wide

case                 
                                               sum                 
enfermedad_text                      1_SIN_SEÑALES 2_ALARMA 3_GRAVE
semana departamento provincia                                      
1      AMAZONAS     UTCUBAMBA                  0.0      1.0     0.0
       ANCASH       CASMA                      2.0      0.0     0.0
       AYACUCHO     HUANTA                     7.0      0.0     0.0
                    LA MAR                    53.0      1.0     0.0
       CAJAMARCA    JAEN                       4.0      0.0     0.0
...                                            ...      ...     ...
52     SAN MARTIN   MARISCAL CACERES           1.0      0.0     0.0
                    MOYOBAMBA                  1.0      0.0     0.0
                    TOCACHE                    4.0      3.0     0.0
       UCAYALI      ATALAYA                    0.0      1.0     0.0
                    CORONEL PORTILLO          19.0     17.0     0.0

[1618 rows x 3 columns]

Para prevenir y también actuar en las provincias, crearemos un status de ALARMA 

In [19]:
sumCases =ByWeekPlace_wide.sum(axis=1)
sumCases

semana  departamento  provincia       
1       AMAZONAS      UTCUBAMBA            1.0
        ANCASH        CASMA                2.0
        AYACUCHO      HUANTA               7.0
                      LA MAR              54.0
        CAJAMARCA     JAEN                 4.0
                                          ... 
52      SAN MARTIN    MARISCAL CACERES     1.0
                      MOYOBAMBA            1.0
                      TOCACHE              7.0
        UCAYALI       ATALAYA              1.0
                      CORONEL PORTILLO    36.0
Length: 1618, dtype: float64

In [20]:
#Creamos el indicador de ALARMA
shareAlarma=ByWeekPlace_wide.loc[:,('case','sum','2_ALARMA')]/sumCases
shareAlarma.name='shareAlarma'
shareAlarma

semana  departamento  provincia       
1       AMAZONAS      UTCUBAMBA           1.000000
        ANCASH        CASMA               0.000000
        AYACUCHO      HUANTA              0.000000
                      LA MAR              0.018519
        CAJAMARCA     JAEN                0.000000
                                            ...   
52      SAN MARTIN    MARISCAL CACERES    0.000000
                      MOYOBAMBA           0.000000
                      TOCACHE             0.428571
        UCAYALI       ATALAYA             1.000000
                      CORONEL PORTILLO    0.472222
Name: shareAlarma, Length: 1618, dtype: float64

In [21]:
shareAlarma=shareAlarma.reset_index()
shareAlarma

,semana,departamento,provincia,shareAlarma
0,1,AMAZONAS,UTCUBAMBA,1.000000
1,1,ANCASH,CASMA,0.000000
2,1,AYACUCHO,HUANTA,0.000000
3,1,AYACUCHO,LA MAR,0.018519
4,1,CAJAMARCA,JAEN,0.000000
...,...,...,...,...
1613,52,SAN MARTIN,MARISCAL CACERES,0.000000
1614,52,SAN MARTIN,MOYOBAMBA,0.000000
1615,52,SAN MARTIN,TOCACHE,0.428571
1616,52,UCAYALI,ATALAYA,1.000000


Busquemos la peor región por provincia en las semanas de acuerdo con nuestro indicador

In [22]:
where = shareAlarma.groupby(['semana','departamento'])['shareAlarma'].idxmax()
worst_prov_week = shareAlarma.loc[where].reset_index(drop=True)
worst_prov_week

,semana,departamento,provincia,shareAlarma
0,1,AMAZONAS,UTCUBAMBA,1.000000
1,1,ANCASH,CASMA,0.000000
2,1,AYACUCHO,LA MAR,0.018519
3,1,CAJAMARCA,SAN IGNACIO,0.500000
4,1,CUSCO,LA CONVENCION,0.058824
...,...,...,...,...
754,52,LORETO,MAYNAS,0.375000
755,52,MADRE DE DIOS,TAHUAMANU,0.000000
756,52,PIURA,SULLANA,0.500000
757,52,SAN MARTIN,TOCACHE,0.428571


In [23]:
len(worst_prov_week.provincia.value_counts())


59

In [24]:
len(worst_prov_week[worst_prov_week.shareAlarma>0].provincia.value_counts()) #limpiamos


51

In [25]:
worst_ProvWeek_alarma=worst_prov_week[worst_prov_week.shareAlarma>0].loc[:,['departamento','provincia']]
worst_ProvWeek_alarma.reset_index(drop=True,inplace=True)
worst_ProvWeek_alarma

,departamento,provincia
0,AMAZONAS,UTCUBAMBA
1,AYACUCHO,LA MAR
2,CAJAMARCA,SAN IGNACIO
3,CUSCO,LA CONVENCION
4,HUANUCO,LEONCIO PRADO
...,...,...
430,CUSCO,LA CONVENCION
431,LORETO,MAYNAS
432,PIURA,SULLANA
433,SAN MARTIN,TOCACHE


In [26]:
indexList=['departamento','provincia']
aggregator={'provincia':['count']}
worst_ProvWeek_alarma_Frequency=worst_ProvWeek_alarma.groupby(indexList,observed=True).agg(aggregator)
worst_ProvWeek_alarma_Frequency

provincia
                                        count
departamento  provincia                      
AMAZONAS      UTCUBAMBA                     8
ANCASH        CASMA                         9
              SANTA                         3
AYACUCHO      HUANTA                        6
              LA MAR                        8
CAJAMARCA     JAEN                         21
              SAN IGNACIO                   1
CUSCO         LA CONVENCION                27
HUANUCO       HUANUCO                       2
              LEONCIO PRADO                20
              PUERTO INCA                   8
ICA           ICA                           2
JUNIN         CHANCHAMAYO                  11
              SATIPO                       23
LA LIBERTAD   ASCOPE                        4
              GRAN CHIMU                    1
              PACASMAYO                     1
              TRUJILLO                     17
              VIRU                          8
LAMBAYEQUE    CHICLAYO                      6
              LAMBAYEQUE                    7
LIMA          LIMA                          2
LORETO        ALTO AMAZONAS                 3
              DATEM DEL MARAÑON             2
              LORETO                       11
              MAYNAS                       26
              PUTUMAYO                      1
              REQUENA                       9
MADRE DE DIOS TAHUAMANU                     7
              TAMBOPATA                    19
PASCO         OXAPAMPA                     10
PIURA         MORROPON                      4
              PAITA                         5
              PIURA                         8
              SECHURA                      18
              SULLANA                      10
              TALARA                        7
SAN MARTIN    BELLAVISTA                    1
              LAMAS                         2
              MARISCAL CACERES              5
              MOYOBAMBA                     7
              PICOTA                        1
              RIOJA                         3
              SAN MARTIN                    5
              TOCACHE                       9
TUMBES        CONTRALMIRANTE VILLAR         4
              TUMBES                       17
              ZARUMILLA                     1
UCAYALI       ATALAYA                      16
              CORONEL PORTILLO             13
              PADRE ABAD                   16

In [27]:
# limpieza final
worst_ProvWeek_alarma_Frequency.columns=['WeekAffected']
worst_ProvWeek_alarma_Frequency=worst_ProvWeek_alarma_Frequency[worst_ProvWeek_alarma_Frequency.WeekAffected>2]
worst_ProvWeek_alarma_Frequency.reset_index(inplace=True)
worst_ProvWeek_alarma_Frequency

,departamento,provincia,WeekAffected
0,AMAZONAS,UTCUBAMBA,8
1,ANCASH,CASMA,9
2,ANCASH,SANTA,3
3,AYACUCHO,HUANTA,6
4,AYACUCHO,LA MAR,8
5,CAJAMARCA,JAEN,21
6,CUSCO,LA CONVENCION,27
7,HUANUCO,LEONCIO PRADO,20
8,HUANUCO,PUERTO INCA,8
9,JUNIN,CHANCHAMAYO,11


In [28]:
#Gráfico d ecantidades en las semanas del 2016
alt_worstProv=alt.Chart(worst_ProvWeek_alarma_Frequency)

enc_worstProv=alt_worstProv.encode(
    y='departamento',
    x='provincia',
    text='WeekAffected:O',
    size='WeekAffected:O'
)

enc_worstProv.mark_text()

alt.Chart(...)

In [29]:
indexList=['semana','provincia','enfermedad_text']
aggregator={'case':['sum']}
ByWeekProv=dengue.groupby(indexList,observed=True).agg(aggregator)
ByWeekProv_wide=ByWeekProv.unstack().fillna(0)
ByWeekProvAlarm=ByWeekProv_wide.loc[:,('case','sum','2_ALARMA')]/ByWeekProv_wide.sum(axis=1)
ByWeekProvAlarm.name='alarmShare'

ByWeekProvAlarm=ByWeekProvAlarm.reset_index()
ByWeekProvAlarm

,semana,provincia,alarmShare
0,1,ASCOPE,0.000000
1,1,ATALAYA,1.000000
2,1,CASMA,0.000000
3,1,CHANCHAMAYO,0.181818
4,1,CHICLAYO,0.000000
...,...,...,...
1613,52,SULLANA,0.500000
1614,52,TAHUAMANU,0.000000
1615,52,TALARA,0.000000
1616,52,TAMBOPATA,0.000000


In [30]:
ByWeekProvAlarm_focus=ByWeekProvAlarm[ByWeekProvAlarm.alarmShare>0]


In [31]:
edges=[-1, .10, .25, .5,1]
theLabels=["a.below10%","b.11-25%","c.26-50%","d.above50%"]
ByWeekProvAlarm_focus.loc[:,"alarmLevels"]=pd.cut(ByWeekProvAlarm_focus['alarmShare'],
                                            include_lowest=True,
                                            bins=edges, 
                                            labels=theLabels,
                                            ordered=True)

##
ByWeekProvAlarm_focus.head()

C:\Users\Nicolas Silva\AppData\Local\Temp\ipykernel_4788\2170656152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ByWeekProvAlarm_focus.loc[:,"alarmLevels"]=pd.cut(ByWeekProvAlarm_focus['alarmShare'],


,semana,provincia,alarmShare,alarmLevels
1,1,ATALAYA,1.000000,d.above50%
3,1,CHANCHAMAYO,0.181818,b.11-25%
5,1,CORONEL PORTILLO,0.200000,b.11-25%
8,1,LA CONVENCION,0.058824,a.below10%
9,1,LA MAR,0.018519,a.below10%


In [32]:
alt_WorstProv=alt.Chart(ByWeekProvAlarm_focus).encode(x='semana:O',
                                                      y=alt.Y('provincia:N',
                                                              sort=alt.EncodingSortField(field='alarmShare',op='max',order='descending')))
enc1_WorstProv=alt_WorstProv.encode(
    color=alt.Color('alarmLevels:O').scale(scheme="set2", reverse=False)
)
enc1_WorstProv.mark_rect()

alt.Chart(...)

In [33]:
enc2_WorstProv=alt_WorstProv.encode(
    text=alt.Text('alarmShare:Q', format=".1f"),
    opacity=alt.condition('datum.alarmShare >= 0.3', alt.value(1), alt.value(0)))
enc2_WorstProv.mark_text(fontStyle='bold')

alt.Chart(...)

Gráfico de semanas en estado de alerta:

In [34]:
enc1_WorstProv.mark_rect() + enc2_WorstProv.mark_text()


alt.LayerChart(...)